# OLD DATASET

In [1]:
# # imports .
# import glob
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Get data file names
# path = r'./data/1_tournaments'
# all_files = glob.glob(path + "/*.csv")


# start_year_considered = 1989
# cols_tournaments = ["Tournament", "Location", "Date", "Surface"]
# cols_tournaments_idxs = [3, 6, 7, 14]

# df_tournaments = pd.DataFrame([], columns = cols_tournaments)

# for filename in all_files:
#     start_year = filename.split("/")[-1].split("_")[1].split("-")[0]
#     if int(start_year) > start_year_considered: 
#         df = pd.read_csv(filename, index_col = None, header = 0)
#         df = pd.DataFrame(df.iloc[:, cols_tournaments_idxs].to_numpy(), columns = cols_tournaments, index = None)
        
#         df_tournaments = pd.concat([df_tournaments, df])

In [2]:
# # PREPROCESSING
# # Format Date as DateTime.
# df_tournaments["Date"] = df_tournaments["Date"].apply(lambda x: x.replace(".", "-"))
# df_tournaments['Date'] =  pd.to_datetime(df_tournaments['Date'])

In [3]:
# # Sort by Date, Reset Index and drop Na
# df_tournaments = df_tournaments.sort_values(by = "Date").reset_index(drop = True).dropna()

In [4]:
# from geopy.geocoders import Nominatim
# # Dictionary of Countries 
# city_and_country = df_tournaments[df_tournaments['Location'].str.contains(",")].Location.unique()

# city_to_country_dict = {}
# for cc in city_and_country:
#     cc_split = cc.split(",")
#     city_to_country_dict[cc_split[0]] = cc_split[1]

# for loc in df_tournaments.Location.unique():
#     if "," in loc:
#         continue
#     geolocator = Nominatim(user_agent = "tennis_predictor")
#     location = geolocator.geocode("", language = "en")
#     print(location)
#     country = location.address.split(",")[-1]
#     city_to_country_dict[loc] = country

# # Standardize Locations to Countries
# df_tournaments["Location"] = \
#     df_tournaments["Location"].apply( \
#         lambda x: city_to_country_dict[x] if (x in city_to_country_dict) else x.split(",")[1] \
#     )

In [5]:
# # Change Carpet to Hard
# df_tournaments["Surface"] = df_tournaments["Surface"].apply(lambda x: x if x != "Carpet" else "Hard") 

# display(df_tournaments)

# NEW DATASET

In [6]:
# imports .
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Get data file names
path = r'./data/atp_matches'
all_files = glob.glob(path + "/*.csv")


start_year_considered = 1989

tournament_old_new_columns = {
   "tourney_name": "Tourney",
    "tourney_date": "Date", 
    "winner_name": "Winner", 
    "winner_age": "WinnerAge", 
    "winner_ioc": "WinnerNationality",
    "loser_name": "Loser", 
    "loser_age": "LoserAge",
    "loser_ioc": "LoserNationality",
    "score": "Score",
    "best_of": "BestOf"
}

df_tournaments = pd.DataFrame([])

for filename in all_files:
    start_year = filename.split("/")[-1].split("_")[-1].split(".")[0]
    if int(start_year) > start_year_considered: 
        df = pd.read_csv(filename, index_col = None)        
        df_tournaments = pd.concat([df_tournaments, df])

df_tournaments = df_tournaments[tournament_old_new_columns.keys()]
df_tournaments.rename(columns = tournament_old_new_columns, inplace = True)

In [7]:
# PREPROCESSING
# Format Date as DateTime.
df_tournaments["Date"] = df_tournaments["Date"].apply(lambda x: str(x)[0:4] + "-" + str(x)[4:6] + "-" + str(x)[6:])
df_tournaments['Date'] =  pd.to_datetime(df_tournaments['Date'])

# Sort by Date, Reset Index and drop Na and Davis/Laver Cup
df_tournaments = df_tournaments.sort_values(by = "Date").reset_index(drop = True).dropna()
df_tournaments = df_tournaments[~(df_tournaments.Tourney.str.contains("Davis"))]
df_tournaments = df_tournaments[~(df_tournaments.Tourney.str.contains("Laver"))]


In [8]:
from geopy.geocoders import Nominatim

## INITIALISE ONLY WHEN RERUNNING FROM SCRATCH
# tournament_to_country_dict = {}
#############################################

# Dictionary of Countries 
for tournament in df_tournaments.Tourney.unique():
           
    if tournament in tournament_to_country_dict:
        continue
    if "Cup" in tournament or "Finals" in tournament:
        tournament_to_country_dict[tournament] = "International"
    
    else: 
        t = tournament.replace("Masters", "").replace("ATP", "").replace("Indoor", "")\
                      .replace("Outdoor", "").replace("Olympics", "").replace("Open", "")
        geolocator = Nominatim(user_agent = "tennis_predictor")
        location = geolocator.geocode(t, language = "en")

        country = location.address.split(",")[-1]
        tournament_to_country_dict[tournament] = country


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [9]:
# Standardize Locations to Countries
df_tournaments["Location"] = df_tournaments["Tourney"].apply(lambda x: tournament_to_country_dict[x])

In [10]:
import country_converter as coco

ioc_to_ios = {
    "ALG": "DZA",  
    "ANG": "AGO",
    "ANT": "ATG", 
    "ARU": "ABW",
    "BAH": "BHS",
    "BRN": "BHR",
    "BAN": "BGD",
    "BAR": "BRB",
    "BIZ": "BLZ",
    "BER": "BMU",
    "BHU": "BTN",
    "BOT": "BWA",
    "IVB": "VGB",
    "BRU": "BRN",
    "BUL": "BGR",
    "BUR": "BFA",
    "CAM": "KHM",
    "CAY": "CYM",
    "CHA": "TCD", 
    "CHI": "CHL",
    "CGO": "COG",
    "CRC": "CRI",
    "CRO": "HRV",
    "DEN": "DNK",
    "ESA": "SLV",
    "GEQ": "GNQ",
    "FIJ": "FJI", 
    "GAM": "GMB",
    "GER": "DEU",
    "GRE": "GRC", 
    "GRN": "GRD", 
    "GUA": "GTM", 
    "GUI": "GIN", 
    "GBS": "GNB", 
    "HAI": "HTI", 
    "HON": "HND", 
    "INA": "IDN", 
    "IRI": "IRN", 
    "KUW": "KWT", 
    "LAT": "LVA", 
    "LIB": "LBN", 
    "LES": "LSO", 
    "LBA": "LBY", 
    "MAD": "MDG", 
    "MAW": "MWI", 
    "MAS": "MYS", 
    "MTN": "MRT", 
    "MRI": "MUS", 
    "MON": "MCO", 
    "MGL": "MNG", 
    "MYA": "MMR", 
    "NEP": "NPL", 
    "NED": "NLD", 
    "NCA": "NIC", 
    "NIG": "NER", 
    "NGR": "NGA", 
    "OMA": "OMN", 
    "PLE": "PSE", 
    "PAR": "PRY", 
    "PHI": "PHL", 
    "POR": "PRT", 
    "PUR": "PRI", 
    "SKN": "KNA", 
    "VIN": "VCT", 
    "SAM": "WSM", 
    "KSA": "SAU", 
    "SEY": "SYC", 
    "SIN": "SGP", 
    "SLO": "SVN", 
    "SOL": "SLB", 
    "RSA": "ZAF", 
    "SRI": "LKA", 
    "SUD": "SDN", 
    "SUI": "CHE", 
    "TPE": "TWN", 
    "TAN": "TZA", 
    "TOG": "TGO", 
    "TGA": "TON", 
    "TRI": "TTO", 
    "UAE": "ARE", 
    "ISV": "VIR", 
    "URU": "URY",
    "VAN": "VUT", 
    "VIE": "VNM", 
    "YUG": "SRB",
    "ZAM": "ZMB", 
    "ZIM": "ZWE"
}

In [11]:
coco_converter = coco.CountryConverter()

df_tournaments["WinnerNationality"] = df_tournaments["WinnerNationality"].apply( \
    lambda x: ioc_to_ios[x] if x in ioc_to_ios else x \
)

df_tournaments["WinnerNationality"] = \
    coco_converter.convert(names = df_tournaments["WinnerNationality"].to_numpy(), to = "name_short")

df_tournaments["LoserNationality"] = df_tournaments["LoserNationality"].apply( \
    lambda x: ioc_to_ios[x] if x in ioc_to_ios else x \
)

df_tournaments["LoserNationality"] = \
    coco_converter.convert(names = df_tournaments["LoserNationality"].to_numpy(), to = "name_short")

display(df_tournaments)

,Tourney,Date,Winner,WinnerAge,WinnerNationality,Loser,LoserAge,LoserNationality,Score,BestOf,Location
0,Wellington,1990-01-01,Thomas Hogstedt,26.2,Sweden,Shuzo Matsuoka,22.1,Japan,5-7 7-6 6-2,3,New Zealand
1,Wellington,1990-01-01,Lars Jonsson,19.5,Sweden,Brett Steven,20.6,New Zealand,7-5 6-4,3,New Zealand
2,Wellington,1990-01-01,Richard Fromberg,19.6,Australia,Olivier Delaitre,22.5,France,7-6 6-1,3,New Zealand
3,Wellington,1990-01-01,Paul Chamberlin,27.7,United States,Jens Woehrmann,22.3,Germany,6-3 6-7 6-0,3,New Zealand
4,Wellington,1990-01-01,Gilad Bloom,22.8,Israel,Magnus Gustafsson,22.9,Sweden,6-1 7-5,3,New Zealand
...,...,...,...,...,...,...,...,...,...,...,...
105356,Us Open,2022-08-29,Grigor Dimitrov,31.2,Bulgaria,Steve Johnson,32.6,United States,6-3 6-2 6-2,5,United States
105357,Us Open,2022-08-29,Lorenzo Musetti,20.4,Italy,David Goffin,31.7,Belgium,3-6 7-5 6-4 3-6 7-6(9),5,United States
105358,Us Open,2022-08-29,Gijs Brouwer,26.4,Netherlands,Adrian Mannarino,34.1,France,6-3 6-4 6-4,5,United States
105359,Us Open,2022-08-29,Daniil Medvedev,26.5,Russia,Arthur Rinderknech,27.1,France,6-2 7-5 6-3,5,United States


In [12]:
def parseScore(score):
    sets_won = 0
    sets_lost = 0
    games_won = 0
    games_lost = 0
    
    sets = score.split(" ")
    for s in sets:
        if not(s):
            break
        games = s.split("-")
        
        games_p1 = games[0]
        games_p2 = games[1][0:2] if len(games[1]) and games[1].isnumeric() > 1 else games[1][0]
        
        if not games_p1.isnumeric() or not games_p2.isnumeric():
            break
        
        games_won += int(games_p1)
        games_lost += int(games_p2)
        if games_p1 > games_p2:
            sets_won += 1
        else:
            sets_lost += 1
    return sets_won, sets_lost, games_won, games_lost

In [13]:
# Remove retirements and walk overs
df_tournaments = df_tournaments[~df_tournaments.Score.str.contains(r'[a-zA-Z]')]


# Parse Scores in Games Won, Games Lost, Sets Won, Sets Lost
df_tournaments["SetsWon"] = df_tournaments["Score"].apply( \
    lambda x: parseScore(x)[0]
)
df_tournaments["SetsLost"] = df_tournaments["Score"].apply( \
    lambda x: parseScore(x)[1]                                                       
)
df_tournaments["GamesWon"] = df_tournaments["Score"].apply( \
    lambda x: parseScore(x)[2]
)
df_tournaments["GamesLost"] = df_tournaments["Score"].apply( \
    lambda x: parseScore(x)[3]                                                       
)
display(df_tournaments)

,Tourney,Date,Winner,WinnerAge,WinnerNationality,Loser,LoserAge,LoserNationality,Score,BestOf,Location,SetsWon,SetsLost,GamesWon,GamesLost
0,Wellington,1990-01-01,Thomas Hogstedt,26.2,Sweden,Shuzo Matsuoka,22.1,Japan,5-7 7-6 6-2,3,New Zealand,2,1,18,15
1,Wellington,1990-01-01,Lars Jonsson,19.5,Sweden,Brett Steven,20.6,New Zealand,7-5 6-4,3,New Zealand,2,0,13,9
2,Wellington,1990-01-01,Richard Fromberg,19.6,Australia,Olivier Delaitre,22.5,France,7-6 6-1,3,New Zealand,2,0,13,7
3,Wellington,1990-01-01,Paul Chamberlin,27.7,United States,Jens Woehrmann,22.3,Germany,6-3 6-7 6-0,3,New Zealand,2,1,18,10
4,Wellington,1990-01-01,Gilad Bloom,22.8,Israel,Magnus Gustafsson,22.9,Sweden,6-1 7-5,3,New Zealand,2,0,13,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105356,Us Open,2022-08-29,Grigor Dimitrov,31.2,Bulgaria,Steve Johnson,32.6,United States,6-3 6-2 6-2,5,United States,3,0,18,7
105357,Us Open,2022-08-29,Lorenzo Musetti,20.4,Italy,David Goffin,31.7,Belgium,3-6 7-5 6-4 3-6 7-6(9),5,United States,3,2,26,27
105358,Us Open,2022-08-29,Gijs Brouwer,26.4,Netherlands,Adrian Mannarino,34.1,France,6-3 6-4 6-4,5,United States,3,0,18,11
105359,Us Open,2022-08-29,Daniil Medvedev,26.5,Russia,Arthur Rinderknech,27.1,France,6-2 7-5 6-3,5,United States,3,0,19,10
